In [1]:
import numpy as np
import keras
from keras import layers
import tensorflow as tf

These codes are based on Watters et al. arXiv:1901.07017 & Oxlei et al. npj Computational Materials (2021) 7:65

In [ ]:
training_data = np.load('training_data.npy') #Training data available on Dryad
augmented_data = np.load('augmented_data.npy')

In [4]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    
class VAE2D(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE2D, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta = 2

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            data_shape = tf.shape(data)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(tf.slice(data, [0,0,0,0],[data_shape[0],data_shape[1],data_shape[2],1]), reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + self.beta*kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def evaluate(self, data):
        z_mean, z_log_var, z = self.encoder.predict(data)
        reconstruction = self.decoder.predict(z)
        data_shape = tf.shape(data)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(tf.slice(data, [0,0,0,0],[data_shape[0],data_shape[1],data_shape[2],1]), reconstruction), axis=(1, 2)
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        return {
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [5]:
latent_dim = 64
kernel_size = 4

encoder_inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(32, kernel_size, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(32, kernel_size, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, kernel_size, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, kernel_size, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 1)]  0                                            
______________________________________________________________________________________

In [6]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(4 * 4 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((4, 4, 64))(x)
x = layers.Conv2DTranspose(64, kernel_size, activation="relu", strides=2, padding="same")(x) #added later
x = layers.Conv2DTranspose(64, kernel_size, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, kernel_size, activation="relu", strides=2, padding="same")(x) #added later
x = layers.Conv2DTranspose(32, kernel_size, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, kernel_size, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              66560     
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 64)          65600     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 64)        65600     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 32)        32800     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 64, 64, 32)        1641

In [7]:
np.random.seed(36)

b_vae2d_aug = VAE2D(encoder, decoder)
b_vae2d_aug.beta = 2

b_vae2d_aug.compile(optimizer=keras.optimizers.Adam())

In [ ]:
history_aug = b_vae2d_aug.fit(np.concatenate((training_data, augmented_data[:,:,:,np.newaxis])), epochs=20)